In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
county_submkt_mapping = pd.read_csv('/mnt/container1/np_forecast_data/submarket_county_mapping.csv')
county_submkt_mapping.head()

,research_market,research_submkt_id,county_geoid,pct_county_area_in_submkt,pct_submkt_area_in_county
0,Phoenix,PHO050,4013,0.045477,1.000000
1,Phoenix,PHO041,4013,0.035820,1.000000
2,Phoenix,PHO063,4013,0.050826,1.000000
3,Phoenix,PHO054,4013,0.074063,0.830409
4,Phoenix,PHO054,4021,0.190402,0.169591


In [3]:
county_submkt_mapping[county_submkt_mapping['research_market']=='Phoenix']['research_submkt_id'].unique()

array(['PHO050', 'PHO041', 'PHO063', 'PHO054', 'PHO056', 'PHO042',
       'PHO038', 'PHO058', 'PHO060', 'PHO049', 'PHO046', 'PHO061',
       'PHO064', 'PHO037', 'PHO052', 'PHO044', 'PHO051', 'PHO062',
       'PHO045', 'PHO040', 'PHO048', 'PHO057', 'PHO059', 'PHO039',
       'PHO043', 'PHO047', 'PHO053', 'PHO055', 'PHO065', 'LIN114'],
      dtype=object)

In [4]:
county_submkt_mapping[county_submkt_mapping['research_market']=='Dallas-Fort Worth']['research_submkt_id'].unique()

array(['DAL031', 'LIN076', 'DAL034-DAL035-DAL041',
       'DAL042-DAL043-DAL044-DAL045-DAL046',
       'DAL050-DAL051-DAL053-DAL054-DAL055',
       'DAL037-DAL038-DAL039-DAL040', 'DAL057-FTW031', 'DAL047-DAL048',
       'DAL052-DAL056', 'DAL049', 'FTW032-FTW033-FTW034-FTW039', 'FTW029',
       'FTW035-FTW036-FTW037-FTW040', 'FTW038-FTW041-FTW042-FTW043'],
      dtype=object)

In [5]:
pho_counties = county_submkt_mapping[(county_submkt_mapping['research_submkt_id']!='LIN114')&(county_submkt_mapping['research_market']=='Phoenix')]['county_geoid'].unique().tolist()
dal_counties = county_submkt_mapping[(county_submkt_mapping['research_submkt_id']!='LIN076')&(county_submkt_mapping['research_market']=='Dallas-Fort Worth')]['county_geoid'].unique().tolist()

In [6]:
pho_counties, dal_counties

([4013, 4021, 4019],
 [48231,
  48213,
  48119,
  48121,
  48139,
  48257,
  48085,
  48113,
  48397,
  48439,
  48497,
  48251,
  48367,
  48221])

In [7]:
def get_county_fips_code(state_fips_code):
    counties_fips_code = []

    if state_fips_code == 4:
        for num in pho_counties:
            if str(num).startswith('4'):
                counties_fips_code.append(str(num)[1:])
    elif state_fips_code == 48:
        for num in dal_counties:
            if str(num).startswith('48'):
                counties_fips_code.append(str(num)[2:])
    return counties_fips_code

In [8]:
pho_fips_codes = get_county_fips_code(4)
dal_fips_codes = get_county_fips_code(48)
pho_fips_codes, dal_fips_codes

(['013', '021', '019'],
 ['231',
  '213',
  '119',
  '121',
  '139',
  '257',
  '085',
  '113',
  '397',
  '439',
  '497',
  '251',
  '367',
  '221'])

In [9]:
pho_fips_codes = ','.join(pho_fips_codes)
dal_fips_codes = ','.join(dal_fips_codes)
pho_fips_codes, dal_fips_codes

('013,021,019', '231,213,119,121,139,257,085,113,397,439,497,251,367,221')

Block Group

STATE+COUNTY+TRACT+BLOCK GROUP

2+3+6+1=12

Block Group 1 in Census Tract 2231 in Harris County, TX

482012231001

In [ ]:
def get_blockgroup_data(
    year: str=None,
    variables: str=None,
    state_fips: str=None,
    county_fips: str=None):

    base_url = 'https://api.census.gov/data'
    dataset = 'acs/acs5'

    if year is None:
        print('Please input variable: year.')
    year = year
    
    if variables is None:
        print('Please input variable: variables.')
    variables = variables
    
    level = 'block%20group'
    
    if state_fips is None:
        print('Please input varaible: state_fips.')

    state_fips = state_fips+'%20'
    
    if county_fips is None:
        print('Please input varaible: county_fips.')
        
    county_fips = county_fips  

    api_key = '2caac992889c23e9fe800fc4a11ffd2248146d2c'  

    url = f'{base_url}/{year}/{dataset}?get={variables}&for={level}:*&in=state:{state_fips}&in=county:{county_fips}&key={api_key}'

    r = requests.get(url)

    try:
        resp = r.json()
        if r.status_code == 200:
            
            data = resp[1:]
            df = pd.DataFrame(data, columns=resp[0])
            df['blockgroup_geoid'] = df['state'] + df['county'] + df['tract'] + df['block group']
            df['blockgroup_geoid'] = df['blockgroup_geoid'].astype(str).apply(lambda x: int(x[1:]) if x.startswith('0') else int(x))
            dt = f'{year}-01-01'
            df['date'] = pd.Series([dt] * df.shape[0])
            df['date'] = pd.to_datetime(df['date'])
            df = df.drop(['state', 'county', 'tract','block group'], axis=1)
            

            return df
        else:
            print("Error occurred while retrieving data.")
    except ValueError: 
        print(f"{year}:Response content is not valid JSON.")

In [ ]:
get_blockgroup_data(
    year='2017',
    variables = 'B01001_001E',
    state_fips = '04',
    county_fips = pho_fips_codes)

In [ ]:
def get_asc_blockgroup_data(year, variables, state_fips, county_fips):
    dfs = pd.DataFrame()

    for yr in year:
        df = get_blockgroup_data(yr, variables, state_fips, county_fips)
        dfs = pd.concat([dfs,df])

 
        
    return dfs




In [ ]:
year = [str(yr) for yr in range(2014,2022)]
variables = 'B01001_001E'
state_fips = '04'
county_fips = pho_fips_codes
pho_df = get_asc_blockgroup_data(year,variables,state_fips,county_fips)
pho_df.head()


In [ ]:
pho_df.shape

In [ ]:
year = [str(yr) for yr in range(2014,2022)]
variables = 'B01001_001E'
state_fips = '48'
county_fips = pho_fips_codes
dal_df = get_asc_blockgroup_data(year,variables,state_fips,county_fips)
dal_df.head()

In [ ]:
pho_fips_codes